In [10]:
import platform
import sys
sys.path.append(f"/opt/qti-aic/dev/lib/{platform.machine()}")
import qaicrt

In [ ]:
import sys
sys.path.append("/opt/qti-aic/examples/apps/qaic-python-sdk/qaic")
import torch

import qaic
import os
import torch
import numpy as np
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [12]:
# Set the path of QPC generated with qaic-exec
qpcPath = '/workspace/vfm_fix_outofrange_fp16_qpc'

In [13]:
vfm_sess = qaic.Session(model_path= qpcPath+'/programqpc.bin', num_activations=1)
vfm_sess.setup()

Here we are setting `num_activations = 1` and `set_size = 1`.
Additionally, you can provide `device_id` as inference parameters. 

Please find more details about the options [here](https://docs.qualcomm.com/bundle/resource/topics/AIC_Developer_Guide/).

In [17]:
# Here we are reading out all the input and output shapes/types
all_pixel_values_shape, all_pixel_values_type = vfm_sess.model_input_shape_dict['all_pixel_values']
patch_attn_mask_shape, patch_attn_mask_type = vfm_sess.model_input_shape_dict['patch_attn_mask']
output_shape, output_type = vfm_sess.model_output_shape_dict['vision_embedding']
print(f'all_pixel_values Input shape {all_pixel_values_shape} and type {all_pixel_values_type}')
print(f'patch_attn_mask Input shape {patch_attn_mask_shape} and type {patch_attn_mask_type}')
print(f'Output shape {output_shape} and type {output_type}')

all_pixel_values Input shape (30, 3, 14, 14448) and type float32
patch_attn_mask Input shape (30, 1, 1032) and type int8
Output shape (30, 64, 3584) and type float32


In [16]:
## Check health of the cards before deploying the inference. 
## Status:Ready indicates that the card is in good health and ready to accept inferences
## Status:Error indicates that the card is not in good health. Please contact the system administrator
!/opt/qti-aic/tools/qaic-util -q | grep -e "Status" -e "QID" -e "Nsp Free" -e "Dram Total"

447.77s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


QID 0
	Status:Ready
	Dram Total:31391744 KB
	Nsp Free:0
QID 1
	Status:Ready
	Dram Total:31391744 KB
	Nsp Free:0
QID 2
	Status:Ready
	Dram Total:31391744 KB
	Nsp Free:16
QID 3
	Status:Ready
	Dram Total:31391744 KB
	Nsp Free:16
QID 4
	Status:Ready
	Dram Total:31391744 KB
	Nsp Free:16
QID 5
	Status:Ready
	Dram Total:31391744 KB
	Nsp Free:16
QID 6
	Status:Ready
	Dram Total:31391744 KB
	Nsp Free:16
QID 7
	Status:Ready
	Dram Total:31391744 KB
	Nsp Free:16
QID 8
	Status:Ready
	Dram Total:31391744 KB
	Nsp Free:16
QID 9
	Status:Ready
	Dram Total:31391744 KB
	Nsp Free:16
QID 10
	Status:Ready
	Dram Total:31391744 KB
	Nsp Free:16
QID 11
	Status:Ready
	Dram Total:31391744 KB
	Nsp Free:16
QID 12
	Status:Ready
	Dram Total:31391744 KB
	Nsp Free:16
QID 13
	Status:Ready
	Dram Total:31391744 KB
	Nsp Free:16
QID 14
	Status:Ready
	Dram Total:31391744 KB
	Nsp Free:16
QID 15
	Status:Ready
	Dram Total:31391744 KB
	Nsp Free:16
QID 16
	Status:Ready
	Dram Total:31391744 KB
	Nsp Free:16
QID 17
	Status:Ready
	Dram

In [ ]:
all_pixel_values = torch.load('/workspace/inputs/all_pixel_values.pkl')
patch_attn_mask = torch.load('/workspace/inputs/patch_attn_mask.pkl')

/tmp/ipykernel_134694/2092300747.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  all_pixel_values = torch.load('/workspace/inputs/all_pixel_values.pkl')
/tmp/ipykernel_1

In [ ]:
# Create a input dictionary for given input.
input_dict = {
    "all_pixel_values": all_pixel_values.numpy().astype(all_pixel_values_type),
    "patch_attn_mask" : patch_attn_mask.numpy().astype(patch_attn_mask_type)
    }
input_dict

{'all_pixel_values': array([[[[-0.25490195, -0.31764704,  0.17647064, ...,  0.24705887,
            0.45882356,  0.58431375],
          [-0.32549018, -0.2235294 ,  0.15294123, ...,  0.26274514,
            0.4431373 ,  0.58431375],
          [-0.35686272, -0.17647058,  0.09019613, ...,  0.26274514,
            0.41176474,  0.5686275 ],
          ...,
          [-0.09019607, -0.05882353, -0.38823527, ...,  0.0196079 ,
           -0.16862744,  0.082353  ],
          [-0.09019607, -0.0745098 , -0.3960784 , ...,  0.14509809,
           -0.16862744,  0.0196079 ],
          [-0.18431371, -0.10588235, -0.3960784 , ...,  0.1686275 ,
           -0.08235294, -0.0745098 ]],
 
         [[-0.19215685, -0.24705881,  0.254902  , ...,  0.37254906,
            0.62352943,  0.8117647 ],
          [-0.26274508, -0.15294117,  0.22352946, ...,  0.3803922 ,
            0.60784316,  0.8039216 ],
          [-0.29411763, -0.10588235,  0.1686275 , ...,  0.3803922 ,
            0.5764706 ,  0.79607844],
        

In [ ]:
# Run the model on Qualcomm Cloud AI 100
output = vfm_sess.run(input_dict)
output.shape

{'vision_embedding': array([[[-1.7460938 , -0.28100586, -0.4387207 , ...,  1.3291016 ,
           0.4580078 ,  0.43139648],
         [-1.3994141 ,  0.83984375,  0.1743164 , ...,  1.8408203 ,
           1.6923828 ,  0.62890625],
         [-0.9941406 , -1.5380859 ,  0.5493164 , ...,  0.98291016,
           1.0703125 , -0.03829956],
         ...,
         [-0.46606445,  0.14196777,  0.0769043 , ...,  0.43310547,
           0.515625  ,  0.28710938],
         [-0.5024414 , -1.4091797 , -0.3149414 , ...,  2.0996094 ,
          -0.6435547 , -0.33618164],
         [-1.1113281 , -1.8867188 ,  1.0888672 , ...,  1.7539062 ,
           0.3449707 , -0.19812012]],
 
        [[-1.2880859 , -0.32788086, -0.6669922 , ...,  1.1943359 ,
           0.8183594 ,  0.23413086],
         [-1.6005859 , -0.3383789 ,  0.64501953, ...,  1.4648438 ,
           2.8730469 ,  0.34106445],
         [-0.9746094 , -0.6689453 ,  0.5048828 , ...,  0.35009766,
           1.3671875 , -0.3125    ],
         ...,
         [-0.

In [ ]:
output.dtype

In [ ]:
output

In [ ]:
torch.save(output['vision_embedding'], '/workspace/outputs/vision_embedding_qpc.pkl')

In [ ]:
# reset the session to release the NSP cores
vfm_sess.reset()